Import libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(0,str(Path.cwd().parent))
from package import *
from datetime import datetime
from package.DandjooId import DandjooId
from shapely.geometry import MultiPolygon
import geopandas as gpd
import matplotlib.pyplot as plt
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import socket
pd.set_option('display.max_rows', None, 'display.max_columns', None, 'display.width', None, 'display.max_colwidth', None)

List all the files in the ibsa blob in a data frame with their filename


In [14]:
# Use the credential csv to create a connection to the Azure blob storage container
creds: pd.DataFrame = pd.read_csv('credential.csv', dtype=str, index_col='item')
cred: str = creds.loc['cred', 'value']
ibsa: str = creds.loc['ibsa', 'value']
ibsa_container: ContainerClient = ContainerClient.from_container_url(ibsa, cred)

In [15]:
# Read the file path to get list of file and their respective filenames and save in variable 'A'
A: pd.DataFrame = read_container(container=ibsa_container, blob_folder='cred.txt4')
A['filename'] = A['path'].apply(lambda x: Path(x).name)
display(A.head())


,path,container_name,filename
0,cred.txt4/02 Data Submission v1.zip,ibsa,02 Data Submission v1.zip
1,cred.txt4/0_0_AvonWheatbelt_DWERSuppliedFile.png,ibsa,0_0_AvonWheatbelt_DWERSuppliedFile.png
2,cred.txt4/0_0_Coolgardie_DWERSuppliedFile.png,ibsa,0_0_Coolgardie_DWERSuppliedFile.png
3,cred.txt4/0_0_DamplierLand_DWERSuppliedFile.png,ibsa,0_0_DamplierLand_DWERSuppliedFile.png
4,cred.txt4/0_0_EsperancePlains_DWERSuppliedFile.png,ibsa,0_0_EsperancePlains_DWERSuppliedFile.png


Read original spreadsheet into dataframe and get their filename

In [16]:
# Read original spreadsheet
B: pd.DataFrame = pd.read_excel('IBSA.xlsx')
#display(B.head())
#print(B.columns)

# Create a filename column in the dataframe to house the file names
B['filename'] = B['URL'].apply(lambda x: Path(x).name)
display(B.head())

,dc:title (50 characters or less),dwc:institutionCode (50 characters or less),dwc:datasetID (confirm we have this already),dwc:projectName (coming with SSD - do not limit field length),dwc:recordedBy (coming with SSD),dwc:rightsHolder,dwc:license (coming with phase 2 Gaia),Doc type,URL,filename
0,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2018-0003,"Level 2 Flora and Vegetation Survey : Mesa B-C, Warramboo BWT, Highway to Tod Bore and Mesa A Extension",MWH Global Pty Ltd,Rio Tinto,CC BY 3.0 AU,Supporting material,https://biocollect.ala.org.au/document/download/2020-10/12_Pilbara_DWERSuppliedFile.png,12_Pilbara_DWERSuppliedFile.png
1,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2018-0003,"Level 2 Flora and Vegetation Survey : Mesa B-C, Warramboo BWT, Highway to Tod Bore and Mesa A Extension",MWH Global Pty Ltd,Rio Tinto,CC BY 3.0 AU,License,https://biocollect.ala.org.au/document/download/2020-10/Metadata%20and%20Licensing%20Statement_updated.pdf,Metadata%20and%20Licensing%20Statement_updated.pdf
2,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2018-0003,"Level 2 Flora and Vegetation Survey : Mesa B-C, Warramboo BWT, Highway to Tod Bore and Mesa A Extension",MWH Global Pty Ltd,Rio Tinto,CC BY 3.0 AU,Report,https://biocollect.ala.org.au/document/download/2020-10/Survey_Report_compressed.pdf,Survey_Report_compressed.pdf
3,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2018-0003,"Level 2 Flora and Vegetation Survey : Mesa B-C, Warramboo BWT, Highway to Tod Bore and Mesa A Extension",MWH Global Pty Ltd,Rio Tinto,CC BY 3.0 AU,Supporting material,https://biocollect.ala.org.au/document/download/2020-10/Survey_Report.txt,Survey_Report.txt
4,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2018-0003,"Level 2 Flora and Vegetation Survey : Mesa B-C, Warramboo BWT, Highway to Tod Bore and Mesa A Extension",MWH Global Pty Ltd,Rio Tinto,CC BY 3.0 AU,Data,https://biocollect.ala.org.au/document/download/2020-10/ESRI%20Shapefile.zip,ESRI%20Shapefile.zip


Assign ids to each project in the dataframe

In [17]:
# Initialize the ID generator
id_generator = DandjooId()
#print(dir(id_generator)) # this will list all attributes and methods of the Dandjooid class, then you can select the best method to generate a unique id

# Create a project_id column and assign IDs to each project in the dataframe
B['project_id'] = B.apply(lambda row: id_generator.new_id(), axis=1) # assign id row-wise

# Display the first five rows
display(B.head())

,dc:title (50 characters or less),dwc:institutionCode (50 characters or less),dwc:datasetID (confirm we have this already),dwc:projectName (coming with SSD - do not limit field length),dwc:recordedBy (coming with SSD),dwc:rightsHolder,dwc:license (coming with phase 2 Gaia),Doc type,URL,filename,project_id
0,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2018-0003,"Level 2 Flora and Vegetation Survey : Mesa B-C, Warramboo BWT, Highway to Tod Bore and Mesa A Extension",MWH Global Pty Ltd,Rio Tinto,CC BY 3.0 AU,Supporting material,https://biocollect.ala.org.au/document/download/2020-10/12_Pilbara_DWERSuppliedFile.png,12_Pilbara_DWERSuppliedFile.png,2024IbJ6cd4fd728
1,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2018-0003,"Level 2 Flora and Vegetation Survey : Mesa B-C, Warramboo BWT, Highway to Tod Bore and Mesa A Extension",MWH Global Pty Ltd,Rio Tinto,CC BY 3.0 AU,License,https://biocollect.ala.org.au/document/download/2020-10/Metadata%20and%20Licensing%20Statement_updated.pdf,Metadata%20and%20Licensing%20Statement_updated.pdf,2024IbJ6cd4fed24
2,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2018-0003,"Level 2 Flora and Vegetation Survey : Mesa B-C, Warramboo BWT, Highway to Tod Bore and Mesa A Extension",MWH Global Pty Ltd,Rio Tinto,CC BY 3.0 AU,Report,https://biocollect.ala.org.au/document/download/2020-10/Survey_Report_compressed.pdf,Survey_Report_compressed.pdf,2024IbJ6cd4ff377
3,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2018-0003,"Level 2 Flora and Vegetation Survey : Mesa B-C, Warramboo BWT, Highway to Tod Bore and Mesa A Extension",MWH Global Pty Ltd,Rio Tinto,CC BY 3.0 AU,Supporting material,https://biocollect.ala.org.au/document/download/2020-10/Survey_Report.txt,Survey_Report.txt,2024IbJ6cd4ff745
4,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2018-0003,"Level 2 Flora and Vegetation Survey : Mesa B-C, Warramboo BWT, Highway to Tod Bore and Mesa A Extension",MWH Global Pty Ltd,Rio Tinto,CC BY 3.0 AU,Data,https://biocollect.ala.org.au/document/download/2020-10/ESRI%20Shapefile.zip,ESRI%20Shapefile.zip,2024IbJ6cd4ffbea


Left join A to B on filename to include matching row from B into A, and display a NaN if no matching name is found

In [18]:
def left_join_on_filename(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """
    Perform a left join on two DataFrames based on the 'filename' column.

    Parameters:
    df1 (pd.DataFrame): The left DataFrame.
    df2 (pd.Dataframe): The right DataFrame.

    Returns:
    pd.DataFrame: The resulting DataFrame after the left join.
    
    """

    result = df1.merge(df2, on='filename', how='left')
    return result

# Assign variable and Call the function
result = left_join_on_filename(A, B)

# Display first five row
display(result.head())

,path,container_name,filename,dc:title (50 characters or less),dwc:institutionCode (50 characters or less),dwc:datasetID (confirm we have this already),dwc:projectName (coming with SSD - do not limit field length),dwc:recordedBy (coming with SSD),dwc:rightsHolder,dwc:license (coming with phase 2 Gaia),Doc type,URL,project_id
0,cred.txt4/02 Data Submission v1.zip,ibsa,02 Data Submission v1.zip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cred.txt4/0_0_AvonWheatbelt_DWERSuppliedFile.png,ibsa,0_0_AvonWheatbelt_DWERSuppliedFile.png,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2018-0024,Habitat Tree Assessment of Proposed Clearing Areas (CPS 7524/2) Wandering-Narrogin Road (~SLK 18.42 to 20.59). Shire of Cuballing,Ecoedge,Ecoedge,CC BY 3.0 AU,Supporting material,https://biocollect.ala.org.au/document/download/2018-11/0_0_AvonWheatbelt_DWERSuppliedFile.png,2024IbJ6cd5135dc
2,cred.txt4/0_0_Coolgardie_DWERSuppliedFile.png,ibsa,0_0_Coolgardie_DWERSuppliedFile.png,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2019-0092,Fauna survey for Mungari Gold Operations Cutter's Ridge Project,Phoenix Environmental Sciences Pty Ltd,Phoenix Environmental Sciences Pty Ltd,CC BY 3.0 AU,Supporting material,https://biocollect.ala.org.au/document/download/2019-12/0_0_Coolgardie_DWERSuppliedFile.png,2024IbJ6cd56a542
3,cred.txt4/0_0_Coolgardie_DWERSuppliedFile.png,ibsa,0_0_Coolgardie_DWERSuppliedFile.png,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2019-0248,Vertebrate Fauna Risk Assessment for Rose South Project Area,Terrestrial Ecosystems,Terrestrial Ecosystems,CC BY 3.0 AU,Supporting material,https://biocollect.ala.org.au/document/download/2020-04/0_0_Coolgardie_DWERSuppliedFile.png,2024IbJ6cd5c04e6
4,cred.txt4/0_0_Coolgardie_DWERSuppliedFile.png,ibsa,0_0_Coolgardie_DWERSuppliedFile.png,Index of Biodiversity Surveys for Assessments IBSA,Dept of Water and Environmental Regulation (DWER),IBSA-2020-0086,"Targeted Vegetation and Flora Survey of the Waihi Gold Operations, Tenements M30/255 & M30/126, October 2019",Jenny Borger Botanical Consulting,Ora Banda Mining,CC BY 3.0 AU,Supporting material,https://biocollect.ala.org.au/document/download/2020-09/0_0_Coolgardie_DWERSuppliedFile.png,2024IbJ6cd5f0ad7


Getting more information about the NAN values

In [24]:
#def count_nan_and_files_with_nan(df: pd.DataFrame) -> None:
    #"""
    #Count the number of NaN values in each column and identify files with NaN values.

    #Parameters:
    #df (pd.DataFrame): The DataFrame to analyze.

    #Returns:
    #None
    
    #"""
    # Count the number of NaN values in each column
    #nan_counts = df.isnull().sum()
    #print("Number of NaN values in each column:")
    #print(nan_counts)

    # Identify rows with NaN values
    #rows_with_nan = df[df.isnull().any(axis=1)]

    # Display the filenames and the count of NaN values in those rows
    #files_with_nan = rows_with_nan['filename']
    #nan_files_count = rows_with_nan.shape[0]

    #print(f"\nNumber of files with NaN values: {nan_files_count}")
    #print("Files with NaN values:")
    #print(files_with_nan)

# Call the function
#count_nan_and_files_with_nan(result)